In [ ]:
from math import cos, sin
from typing import List
 
import matplotlib.pyplot as plt

from numpy import array, cross, dot, identity, linspace, pi, stack,  zeros
from numpy.linalg import norm
from numpy.typing import NDArray
from pyquaternion import Quaternion
from scipy.linalg import expm
from scipy.spatial.transform import Rotation, Slerp

# Rotation matrices

## Rotation matrix factory method

In this example, we instantiate an identity rotation I from a rotation matrix.

* The `from_matrix()` method allows the instantiation of a `Rotation` object from a NumPy array. The array must have shape `(3, 3)`.

* The `as_matrix()` method returns a representation of the rotation as a NumPy array (the rotation matrix). It can be further manipulated as NumPy array, but it will no longer support any specific `Rotation` operators.

In [ ]:
I = array([(1.0, 0.0, 0.0), (0.0, 5.0, 0.0), (0.0, 0.0, 1.0)])
R = Rotation.from_matrix(I)
R

## Working with multiple rotations

The `Rotation` object in SciPy does not just represent a single rotation - it can also be used to represent `N` separate rotations that share a common function - e.g. they can be applied in bulk.

Most methods will accept a single NumPy array of shape `(3, )` (for vectors) or `(3, 3)` (for matrices), or a NumPy array of shape `(M, shape)`, where `M` is the number of rotations.

## Applying rotations to vectors

We can apply a rotation to a vector by using the `apply()` method. As the identity rotation does not transform the original vector, when applied to a vector `v`, its components are not altered.

In [ ]:
v = array([1.0, 1.0, 1.0])
R.apply(v)

The result of the function `apply()` is a NumPy array representing the transformed vector. We can apply a rotation to many vectors in a single function call by passing an array of shape `(N, 3)` to speed up large computations.

In [ ]:
vv = array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]])
R.apply(vv)

## Multiplication using rotation matrices

In this example, we multiply two rotation matrices to concatenate two rotations.



In [ ]:
# This is a rotation of positive 90 degrees around the Z axis.
R_A = Rotation.from_matrix(array([[0.0, -1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 1.0]]))


# This is a rotation of negative 90 degrees around the Y axis.
R_B = Rotation.from_matrix(array([[0.0, 0.0, -1.0], [0.0, 1.0, 0.0], [1.0, 0.0, 0.0]]))

R_AB = R_A * R_B
R_AB.as_matrix()

The composition of two rotations is equivalent to the matrix multiplication `A @ B` using NumPy arrays for the rotation matrices, but it preserves the `Rotation` type, such that further operations can be performed on the resulting rotation.

We can verify the result of the rotation by performing the matrix multiplication 'manually'.

In [ ]:
M_AB = R_A.as_matrix() @ R_B.as_matrix()
M_AB

Notice that, just like matrix multiplication, `Rotation` objects do not commute!

In [ ]:
R_B * R_A == R_A * R_B

# Axis-angle representations and exponential map

SciPy provide an axis-angle representation through the methods and members ending with `rotvec` (remember, these are singular around $\pi$). It also allows direct instantiations and conversion to and from modified Rodrigues parameters (always be mindful of singularities around $2 \pi$):

In [ ]:
# Instantiate an example rotation one-degree rotation around a generic axis
R : NDArray = Rotation.from_rotvec(array([0.86, 0.31, 0.12]), degrees=1.0)

v_rv : NDArray = R.as_rotvec()
v_mrp : NDArray = R.as_mrp()

print(f'R as axis-angle:\t\t\t{v_rv}\nR as modified Rodrigues parameters:\t{v_mrp}')

We can retrieve the axis and angle directly from the rotation vector using the accessory method `as_rotvec`, which returns a vector that is **NOT** normalized!

In [ ]:
axis : NDArray = R.as_rotvec()
angle : float = norm(axis)

print(f'Axis:\t\t\t{axis}\nAngle:\t\t\t{angle}\nNormalized vector:\t{axis / norm(axis)}')

## Applying rotations using the Euler-Rodrigues formula

In this section, we apply the Euler-Rodrigues formula and compare to the canonical rotation performed by SciPy. The example also shows how to generate a random rotation matrix which guarantees a normally-distributed orthogonal tensor.

In [ ]:
# Define a convenience function for the Euler-Rodrigues formula
def euler_rodrigues(k : NDArray, theta: float, v: NDArray) -> NDArray:
    return v * cos(theta) + cross(k,v) * sin(theta) + k * dot(k, v) * (1 - cos(theta))

# Instantiate a random rotation
random_rotation = Rotation.random()
axis : NDArray = random_rotation.as_rotvec()
angle : float = norm(axis)

v : NDArray = array([0.43, 5.43, 0.11])

v_ed = euler_rodrigues(axis / angle, angle, v)
print(f'According to Euler-Rodrigues formula:\t{v_ed}\nAccording to SciPy:\t\t\t{random_rotation.apply(v)}')


## Rotations using the exponential map

We can use the definition of skew-symmetric matrix and the exponential map to find a perfectly equivalent result as in the previous section.

In [ ]:
# Define a convenient function the skew-symmetric matrix
def skew(v : NDArray) -> NDArray:
    return cross(v, -1.0 * identity(v.shape[0]))

v_skew : NDArray = skew(axis / angle)

print(f'The skew symmetric matrix for vector v is:\n{v_skew}\n')

v_em = Rotation.from_matrix(expm(angle * v_skew)).apply(v)
print('Applying R to v yields ...')
print(f'According to the exponential map:\t{v_em}\nAccording to SciPy:\t\t\t{random_rotation.apply(v)}')

# Quaternions

## Quaternion instantiation

Quaternions in `pyquaternion` can be initalized from:

* A single scalar (for Quaternions representing real numbers)
* Python iterables (e.g. a list or NumPy array)
* Elements in the order (w, x, y, z)
* A scalar and a real component
* An axis and an angle
* A rotation matrix 

In [ ]:
# Scalar initialization
Quaternion(0.0)

# Iterable initializations
Quaternion((1.0, 0.0, 0.0, 0.0))        # Tuple
Quaternion([1.0, 0.0, 0.0, 0.0])        # List
Quaternion(array([1.0, 0.0, 0.0, 0.0])) # NumPy array

# Element initialization
Quaternion(1.0, 0.0, 0.0, 0.0)
Quaternion(*(1.0, 0.0, 0.0, 0.0))

# Scalar and real component initialization
Quaternion(scalar=1.0, vector=(0.0, 0.0, 0.0))   # Vector can be any iterable

# Axis-angle initialization
Quaternion(axis=(1.0, 0.0, 0.0), angle=pi/2)
Quaternion(axis=(1.0, 0.0, 0.0), degrees=90)

# Rotation matrix initialization
Quaternion(matrix=array([[1.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0],
                         [0.0, 0.0, 1.0]]))


q1 : Quaternion = Quaternion(0.0, 2.0, 3.0, 4.0)
q2 : Quaternion = Quaternion(2.0, 3.0, 4.0, 5.0)

## Quaternion algebra

The sum of two quaternions is just the sum of their components:

In [ ]:
q1 + q2

The multiplication operator automatically follows the quaternion algebra multiplication rules:

In [ ]:
q2 * q1

We can invert quaternions using the inverse member. Notice that this is precomputed during instantiation.

In [ ]:
q1.inverse

We can compute the quaternion conjugate using the `conjugate` member. Keep in mind this is equal to the inverse only for unit quaternions, such as `q1` in this example!

In [ ]:
q1.conjugate

If a quaternion is not unit, we can normalize it using either the `unit` or the `normalised` properties. Notice that these properties return a copy of the quaternion!

In [ ]:
q1.unit == q1.normalised

## Conversion to other representations

We can convert quaternion directly to rotation matrices by accessing the `rotation_matrix` member:

In [ ]:
q1.rotation_matrix

Notice how this method returns a NumPy array, which can be fed directly into SciPy for interoperability between the two libraries (at the cost of expressing the rotation matrix):

In [ ]:
Rotation.from_matrix(q1.rotation_matrix)

SciPy can also accept quaternions directly from PyQuaternion, although we need to get a bit more creative with the syntax to ensure proper type conversion:

In [ ]:
Rotation.from_quat([*q1.vector, q1.scalar])

We may also access the axis-angle representation of the quaternion through the members `axis` and `angle`: remember this is not immune from singualarities!

In [ ]:
print(f'Axis:\t{q1.axis}\nAngle:\t{q1.angle}')

## Rotating vectors using quaternions

To rotate a vector, we can use the `rotate(vector)` method, or we can use the rotation expression as described in the slides.

In [ ]:
# Here we use the 'normalised' property to ensure we are working with a unit
# quaternion which actually represents a rotation.
v = array([0.24, 0.11, 0.94])
q = Quaternion(1.0, 0.21, 0.33, 0.65).normalised

# Notice how we "expand" the vector a quaternion with null 'w' component to
# perform the multiplication, then we retrieve only the vector part.
p_auto : NDArray = q.rotate(v)
p_manual : NDArray = (q * Quaternion([0.0, *v]) * q.conjugate).vector
print(f'Using quaternion multiplication\tp = {p_manual}')
print(f'Using the method \'rotate\'\tp = {p_auto}')

## Comparison with Rodrigues' rotation formula

In [ ]:
v = array([0.65, 0.31, 0.99])
q = Quaternion(0.56, 0.78, 0.55, 0.15).normalised

v_quaternion = q.rotate(v)

theta = q.angle
k = q.axis
v_rodrigues = euler_rodrigues(k, theta, v)

print(f'Using quaternion multiplication:\tV = {v_quaternion}\nUsing Rodrigues formula:\t\tV = {v_rodrigues}')

## Numerical integration and differentiation

We can integrate a quaternion over time using the `integrate(angular_speed, timestep)` method. Notice how this method performs a single step and updates the quaternion in place. The method assumes constant angular velocity and performs a second-order approximation.

In [ ]:
# Integrate a unit quaternion over 400 timesteps until an angle of 180
# degrees is covered by the rotation.
q = Quaternion()
[q.integrate([pi / 4.0, 0.0, 0.0], 0.01) for time_step in range(0,400)]
q == Quaternion(axis=[1.0, 0.0, 0.0], angle=pi)

The method `derivative(angular_speed)` returns the differential for a single time step:

In [ ]:
# Compute the instantaneous rate of change for the rotation integrated in the
# cell above.
q = Quaternion()
q.derivative([pi/4.0, 0.0, 0.0])

Notice this result matches the formula for the differentiation in the slides (take care of normalising to a unit quaternion and properly offsetting the real part of the vector $\omega$):

$$
\dot{\bm{q}} (t) = \frac{1}{2} \bm{\omega} \bm{q} (t)
$$

In [ ]:
Quaternion(0.5 * array([0.0, pi/4.0, 0.0, 0.0]) * q.normalised)

# Plotting rotations

Below is a script to plot rotations from SciPy as simple coordinate systems in three dimensions.

In [ ]:
def plot_rotation(axes, rotation, label=None, offset=(0.0, 0.0, 0.0), scale=1.0):

    COLORS : List[str] = ('#b50015', '#6f8a32', '#2a7ea0')
    position = array([offset, offset])

    for i, (axis, color) in enumerate(zip((axes.xaxis, axes.yaxis, axes.zaxis), COLORS)):
        axlabel = axis.axis_name
        axis.set_label_text(axlabel)

        # Set the colors
        axis.label.set_color(color)
        axis.line.set_color(color)
        axis.set_tick_params(colors=color)

        line = zeros((2, 3))
        line[1, i] = scale
        line_rot = rotation.apply(line)
        line_plot = line_rot + position
        axes.plot(line_plot[:, 0], line_plot[:, 1], line_plot[:, 2], color)
        text_position = rotation.apply(line[1] * 1.2) + position[0]
        axes.text(*(text_position + position[0]), axlabel.upper(), color=color, va='center', ha='center')

    axes.text(
        *(offset),
        label,
        color='k',
        va='center',
        ha='center',
        bbox={'fc': 'w', 'alpha': 0.8, 'boxstyle': 'ellipse'},
    )

We can use the function above to plot a simple comparison between intrinsic and extrinsic Euler sequences in SciPy:

In [ ]:
rotation : Rotation = Rotation.from_rotvec(array([1.0, 0.0, 0.0]))
sequence : str = 'zyx'
magnitudes : List[float] = [90.0, -30.0, 0.0]    

# Perform an intrinsic and an extrinsic rotation. Notice the use of the
# functions 'lower' and 'upper' to specify the rotation type:
# > sequence.upper() -> 'ZYX' = Intrinsic
# > sequence.lower() -> 'zyx' = Extrinsic
initial_pose : Rotation = rotation.identity()
rotated_pose_intrinsic = rotation.from_euler(sequence.upper(), magnitudes, degrees=True)
rotated_pose_extrinsic = rotation.from_euler(sequence.lower(), magnitudes, degrees=True)

# Plot the rotations with a fixed offset from each other
axes = plt.figure().add_subplot(projection='3d', proj_type='ortho')
plot_rotation(axes, initial_pose, label='$R_1$', offset=(0, 0, 0))
plot_rotation(axes, rotated_pose_intrinsic, label='$R_I$', offset=(3, 0, 0))
plot_rotation(axes, rotated_pose_extrinsic, label='$R_E$', offset=(6, 0, 0))

## Singularities in Euler angles rotations

Singular configurations are a risk when using Euler angles. The singularity depends on the choice of Euler angles sequence, and it is an unavoidable shortcoming of all three-parameters representations.

In [ ]:
singular_rotation = R.from_rotvec([0.0, pi / 2.0, 0.0])
singular_rotation.as_euler('xyz', degrees=False)

# Spherical linear interpolation

SciPy provides a SLERP function that performs quaternion SLERP on a collection of `Rotation` objects.

The collection of input rotations can be thought of as keyframes of an animation of a rotating object. We would like to interpolate between keyframes to provide a smooth transition between different orientation.

In [ ]:
# Instantiate the intiial and final rotations
initial_rotation = R.from_rotvec([0.0, 0.0, 0.0], degrees=0)
final_rotation = R.from_euler('xyz', [90.0, 25.0, 14.0], degrees=True)

# We collect the initial and final rotations in a single (2, 3, 3) array
rotation_matrices = stack([initial_rotation.as_matrix(),
                           final_rotation.as_matrix()])

# We instantiate a SciPy rotation from the array of rotation matrices
key_rotations = Rotation.from_matrix(rotation_matrices)
key_times = linspace(0, 1, len(rotation_matrices))

# Instantiate the Slerp interpolation object
slerp = Slerp(key_times, key_rotations)

# Calculate the interpolated rotations
desired_interpolations : int = 5
key_steps = linspace(0, 1, desired_interpolations)
interpolated_rotations = slerp(key_steps)
interpolated_rotations

We can then plot the results using the `plot_rotation` function in a loop:

In [ ]:
# Create a new figure
axes = plt.figure().add_subplot(projection='3d', proj_type='ortho')

# Plot the initial rotaiton
plot_rotation(axes, initial_rotation, 1, [0.0, 0.0, 0.0])

# Define the offsets for the interpolations and plot the interpolated rotations
offset : float = 5.0
offsets = [[offset * step, 0.0, 0.0] for step, _ in enumerate(key_steps)]
for index, (current_rotation, current_offset) in enumerate(zip([*interpolated_rotations], offsets[1:])):
    plot_rotation(axes, current_rotation, index + 2, current_offset)

# Plot the final rotation
plot_rotation(axes, final_rotation, len(key_steps) + 1, (offsets[-1][0] + offset, 0.0, 0.0))